# 1. QUBO on an open geometry with positive coefficients

Here we consider the following QUBO problem
$
H = J_1 \sum_ix_i + J_2 \sum_{ij}x_ix_j
$
where $J_2>0$ and $J_1$ could be tuned to be either positive or negative. The binary variables $x_i$ correspond to the atoms in the graphene with open geometry. 

The first step is to specify the geometry as a list of neighbors (here we use 2x2 cells with 8 atoms as an example)

In [447]:
neighbors = [
    (0, 1), (1, 2), (1, 3), (2, 4), (3, 5), (4, 6), (5, 6), (6, 7)
]

Next, we define the QUBO problem

In [258]:
import functools as ft
import numpy as np

def tensor(N, indices):
    """Return the tensor product of a set of binary variables
    
    Example 1: 
        tensor(2, [0, 1]) = array([0, 0, 0, 1]) represents x0x1 with N=2 variables
    
    Example 2: 
        tensor(3, [0, 1]) = array([0, 0, 0, 0, 0, 0, 1, 1]) represents x0x1 with N=3 variables    
        
    Example 3: 
        tensor(3, [0, 2]) = array([0, 0, 0, 0, 0, 1, 0, 1]) represents x0x2 with N=3 variables    
        
    Example 4: 
        tensor(3, [1]) = array([0, 0, 1, 1, 0, 0, 1, 1]) represents x1 with N=3 variables    
        
    Example 5: 
        tensor(3, [0,1,2]) = array([0, 0, 0, 0, 0, 0, 0, 1]) represents x0x1x2 with N=3 variables    
        
    """
    
    list_binary_variables = [(1, 1) for _ in range(N)]
    for ind in indices:
        list_binary_variables[ind] = (0, 1)
        
    return ft.reduce(np.kron, list_binary_variables)

def QUBO(neighbors, J1=0.080403, J2=0.019894):
    J1 = float(J1)
    J2 = float(J2)
    num_atoms = max(list(sum(neighbors, ()))) + 1
    
    H = 0
    # Add the linear tearm
    for i in range(num_atoms):
        H += J1 * tensor(num_atoms, [i])    
        
    # Add the quadratic term
    for (i,j) in neighbors:
        H += J2 * tensor(num_atoms, [i, j])
    
    min_val = min(H)
    min_val_indices = [i for i in range(len(H)) if H[i]==min_val]
    configs = [f'{index:0{num_atoms}b}' for index in min_val_indices]
    
    return H, (min_val, min_val_indices), configs    
    
    

We test several values for J1, and it seems like `QUBO` all gives the correct results.

In [448]:
H, (min_val, min_val_indices), configs = QUBO(neighbors, J1=-0.05)
min_val, configs

(-0.26021199999999994, ['10111101'])

Now we embed the QUBO to the Rydberg Hamiltonian shown in this [example notebook](https://github.com/amazon-braket/amazon-braket-examples/blob/main/examples/analog_hamiltonian_simulation/00_Introduction_of_Analog_Hamiltonian_Simulation_with_Rydberg_Atoms.ipynb). Particularly, we are interested in mapping the QUBO to the following Rydberg Hamiltonian with nonzero global detuning

\begin{align}
H_\text{ryd} = -\Delta_\text{global}\sum_kn_k + \sum_{j=1}^{N-1}\sum_{k=1}^{N}\frac{C_6}{R_{jk}^6}n_jn_k
\end{align}

When $x_j=1$, the corresponding atom will be excited to the Rydberg state, i.e., $n_j=1$. Similarly, $x_j=0$ corresponds to $n_j=0$. 

Let us now determine the atomic distance $R_{jk}$. Suppose we use the maximum allowed detuning $\Delta_\text{global}^\text{max}$, then we have
\begin{align}
\frac{J_1}{J_2} = \frac{\Delta_\text{global}^\text{max}}{C_6/R^6}
\end{align}
which give
\begin{align}
R = \left(\frac{C_6}{\Delta_\text{global}^\text{max}}\frac{J_1}{J_2}\right)^{1/6}
\end{align}
Typically, we have $\Delta_\text{global}^\text{max}=1.25\times10^8$ rad/s and $C_6 = 5.42\times10^{-24} \text{rad m}^6/s$.

We note that we only consider the nearest neighbor (NN) interaction here, because the next-nearest neighbor (NNN) interaction is negligible. To see that, we note the distance between NNN atoms are $\sqrt{3}$ times the distance between the NN atoms, and hence the interaction between the NNN atoms is $1/27$ times the interaction between the NN atoms. 

In [464]:
from braket.ahs.atom_arrangement import AtomArrangement
from braket.timings.time_series import TimeSeries
from braket.ahs.driving_field import DrivingField
from braket.ahs.analog_hamiltonian_simulation import AnalogHamiltonianSimulation

from braket.analog_hamiltonian_simulator.rydberg.constants import (
    RYDBERG_INTERACTION_COEF,
    SPACE_UNIT,
    TIME_UNIT,
)

from braket.analog_hamiltonian_simulator.rydberg.rydberg_simulator_helpers import (
    get_blockade_configurations,
    _get_sparse_ops,
    _get_coefs
)

from braket.analog_hamiltonian_simulator.rydberg.rydberg_simulator_unit_converter import (
    convert_unit,
)

import warnings


def QUBO_ryd(coords, 
             Delta_max=1.25e8, 
             J1=0.080403, 
             J2=0.019894,
             C6 = 5.42e-24,
             threshold_factor = 1/18
            ):
    
    """
    The coords are scaled such that the distance of NN atoms are 1
    The threshold_factor is defined as following
    
        if J2 * threshold_factor > J1, then we will need define R in a different way
    """
    
    J1 = float(J1)
    J2 = float(J2)
    assert J2 > 0 
    
    if J1 == 0:
        Delta_max = 0
#         R = 8e-6
    elif J1 < 0:
        Delta_max = abs(Delta_max)
    else:
        Delta_max = -abs(Delta_max)
        
    if J2 * threshold_factor < abs(J1):
        a = 1
        R = a * (C6/Delta_max * -J1/J2)**(1/6)
    else:
        warnings.warn("J1 is too small")
        R1 = (C6 / 27 / abs(Delta_max))**(1/6)
        R2 = (C6 / abs(Delta_max))**(1/6)        
        R = (R1+R2)/2
        
#     a = 1
#     R = a * (C6/Delta_max * -J1/J2)**(1/6)

#     a = 2.5
#     R = a * (C6/Delta_max * -J1/J2)**(1/6)


#     print(f"R={R}, V={C6/(R**6)}, Delta_max = {Delta_max}")
#     print(f"R={R}, R1={R1}, R2={R2}, V={C6/(R**6)}, Delta_max = {Delta_max}")
    
    
    
    num_atoms = len(coords)
    
    # We will define the Hamiltonian via a ficticious AHS program
    
    # Define the register 
    register = AtomArrangement()

    for coord in coords:
        register.add(np.array(coord) * R)
        
    # Define a const driving field with zero Rabi frequency, and 
    # max allowed detuning
    t_max = 4e-6
    Omega = TimeSeries().put(0.0, 0.0).put(t_max, 0.0)
    Delta = TimeSeries().put(0.0, Delta_max).put(t_max, Delta_max)
    phi = TimeSeries().put(0.0, 0.0).put(t_max, 0.0)
    
    drive = DrivingField(
        amplitude=Omega,
        phase=phi,
        detuning=Delta
    )
    
    program = AnalogHamiltonianSimulation(
        hamiltonian=drive,
        register=register
    )
    
    
    # Now extract the Hamiltonian as a matrix from the program
    
    program = convert_unit(program.to_ir())
        
    configurations = get_blockade_configurations(program.setup.ahs_register, 0.0)

    
    rydberg_interaction_coef = RYDBERG_INTERACTION_COEF / ((SPACE_UNIT**6) / TIME_UNIT)
        
    rabi_ops, detuning_ops, interaction_op, local_detuning_ops = _get_sparse_ops(
        program, configurations, rydberg_interaction_coef
    )
        
    t_max_converted = program.hamiltonian.drivingFields[0].amplitude.time_series.times[-1]
    rabi_coefs, detuning_coefs, local_detuing_coefs = _get_coefs(program, [0, t_max_converted])
    
#     print(f"interaction_op={interaction_op}")
#     print(f"detuning_ops[0]={detuning_ops[0]}")
#     print(f"detuning_coefs[0][-1]={detuning_coefs[0][-1]}")
    
    H = interaction_op - detuning_ops[0] * detuning_coefs[0][-1]
    
    # H is diagonal
    diagH = H.diagonal()
        
    min_val = min(diagH)
    min_val_indices = [i for i in range(len(diagH)) if diagH[i]==min_val]
    configs = [f'{index:0{num_atoms}b}' for index in min_val_indices]
    
    return diagH, (min_val, min_val_indices), configs        
    

### For 2x2 cells with 8 atoms

In [472]:
coords = [[0, 0], 
          [0, -1],
          [-np.sqrt(3)/2, -3/2],
          [np.sqrt(3)/2, -3/2],
          [-np.sqrt(3)/2, -5/2],
          [np.sqrt(3)/2, -5/2],
          [0, -3],
          [0, -4],
         ]

neighbors = [
    (0, 1), (1, 2), (1, 3), (2, 4), (3, 5), (4, 6), (5, 6), (6, 7)
]


# J1 = -0.00001 # a = 3
# J1 = -0.0001 # a = 2
# J1 = -0.001 # a = 1.5
J1 = -0.01 # ? 
# J1 = -0.1 # a = 1

# J1 = -0.0011

diagH, (min_val, min_val_indices), configs = QUBO_ryd(coords, J1=J1)
print(min_val, configs)

H, (min_val, min_val_indices), configs = QUBO(neighbors, J1=J1)
print(min_val, configs)

(-476.18337108470786+0j) ['10011001', '10100101']
-0.04 ['01001101', '10001101', '10011001', '10100101', '10110001', '10110010']


In [445]:
J1range = [-1/i for i in np.linspace(10, 1e2, 100)]

for J1 in J1range:
    
    _, _, configs_1 = QUBO_ryd(coords, J1=J1)

    _, _, configs_2 = QUBO(neighbors, J1=J1)

    configs_3 = [config for config in configs_1 if config in configs_2]
    print(f"len(configs_1), len(configs_2), len(configs_3) = {len(configs_1)}, {len(configs_2)}, {len(configs_3)}, J1={J1}")

len(configs_1), len(configs_2), len(configs_3) = 1, 1, 1, J1=-0.1
len(configs_1), len(configs_2), len(configs_3) = 1, 1, 1, J1=-0.09166666666666667
len(configs_1), len(configs_2), len(configs_3) = 1, 1, 1, J1=-0.08461538461538462
len(configs_1), len(configs_2), len(configs_3) = 1, 1, 1, J1=-0.07857142857142857
len(configs_1), len(configs_2), len(configs_3) = 1, 1, 1, J1=-0.07333333333333333
len(configs_1), len(configs_2), len(configs_3) = 1, 1, 1, J1=-0.06875
len(configs_1), len(configs_2), len(configs_3) = 1, 1, 1, J1=-0.06470588235294118
len(configs_1), len(configs_2), len(configs_3) = 1, 1, 0, J1=-0.06111111111111111
len(configs_1), len(configs_2), len(configs_3) = 1, 1, 1, J1=-0.05789473684210526
len(configs_1), len(configs_2), len(configs_3) = 1, 1, 1, J1=-0.05500000000000001
len(configs_1), len(configs_2), len(configs_3) = 1, 1, 1, J1=-0.05238095238095238
len(configs_1), len(configs_2), len(configs_3) = 1, 1, 1, J1=-0.05
len(configs_1), len(configs_2), len(configs_3) = 1, 1, 1, J

### For 3x3 cells with 18 atoms

In [470]:
coords = [[0, 0], 
          [0, -1],
          [-np.sqrt(3)/2, -3/2],
          [np.sqrt(3)/2, -3/2],
          [-np.sqrt(3)/2, -5/2],
          [np.sqrt(3)/2, -5/2],
          [-np.sqrt(3), -3],
          [0, -3],
          [np.sqrt(3), -3],
          [-np.sqrt(3), -4],
          [0, -4],
          [np.sqrt(3), -4],
          [-np.sqrt(3)/2, -9/2],
          [np.sqrt(3)/2, -9/2],
          [-np.sqrt(3)/2, -10/2],
          [np.sqrt(3)/2, -10/2],                    
          [0, -6],
          [0, -7],
         ]

neighbors = [
    (0, 1), 
    (1, 2), (1, 3), 
    (2, 4), (3, 5), 
    (4, 6), (4, 7), (5, 7), (5, 8),
    (6, 9), (7, 10), (8, 11),
    (9, 12), (10, 12), (10, 13), (11, 13),
    (12, 14), (13, 15),
    (14, 16), (15, 16),
    (16, 17)
]


# J1 = -0.00001 # a = 3
# J1 = -0.0001 # a = 2
# J1 = -0.001 # a = 1.5
# J1 = -0.01 # ? 
J1 = -0.1 # a = 1

# J1 = -0.0011

H, (min_val, min_val_indices), configs = QUBO(neighbors, J1=J1)
print(min_val, configs)


diagH, (min_val, min_val_indices), configs = QUBO_ryd(coords, J1=J1)
print(min_val, configs)

-1.3822259999999988 ['111111111111111111']
(-1625.0178575909356+0j) ['111111111111001111']


In [471]:
J1range = [-1/i for i in np.linspace(10, 1e3, 100)]

for J1 in J1range:
    
    _, _, configs_1 = QUBO_ryd(coords, J1=J1)

    _, _, configs_2 = QUBO(neighbors, J1=J1)

    configs_3 = [config for config in configs_1 if config in configs_2]
    print(f"len(configs_1), len(configs_2), len(configs_3) = {len(configs_1)}, {len(configs_2)}, {len(configs_3)}, J1={J1}")

len(configs_1), len(configs_2), len(configs_3) = 1, 1, 0, J1=-0.1
len(configs_1), len(configs_2), len(configs_3) = 1, 1, 0, J1=-0.05
len(configs_1), len(configs_2), len(configs_3) = 1, 1, 0, J1=-0.03333333333333333
len(configs_1), len(configs_2), len(configs_3) = 1, 1, 0, J1=-0.025
len(configs_1), len(configs_2), len(configs_3) = 1, 1, 0, J1=-0.02
len(configs_1), len(configs_2), len(configs_3) = 1, 20, 1, J1=-0.016666666666666666
len(configs_1), len(configs_2), len(configs_3) = 1, 20, 1, J1=-0.014285714285714285
len(configs_1), len(configs_2), len(configs_3) = 1, 20, 1, J1=-0.0125
len(configs_1), len(configs_2), len(configs_3) = 1, 20, 1, J1=-0.011111111111111112
len(configs_1), len(configs_2), len(configs_3) = 1, 20, 1, J1=-0.01
len(configs_1), len(configs_2), len(configs_3) = 1, 20, 1, J1=-0.00909090909090909
len(configs_1), len(configs_2), len(configs_3) = 1, 20, 1, J1=-0.008333333333333333
len(configs_1), len(configs_2), len(configs_3) = 1, 20, 1, J1=-0.007692307692307693
len(confi

/var/folders/17/wqjdf0nn4dj11p1h8yrx44100000gr/T/ipykernel_82855/3077496220.py:56: UserWarning: J1 is too small
  warnings.warn("J1 is too small")


len(configs_1), len(configs_2), len(configs_3) = 1, 20, 1, J1=-0.001098901098901099
len(configs_1), len(configs_2), len(configs_3) = 1, 20, 1, J1=-0.0010869565217391304
len(configs_1), len(configs_2), len(configs_3) = 1, 20, 1, J1=-0.001075268817204301
len(configs_1), len(configs_2), len(configs_3) = 1, 20, 1, J1=-0.0010638297872340426
len(configs_1), len(configs_2), len(configs_3) = 1, 20, 1, J1=-0.0010526315789473684
len(configs_1), len(configs_2), len(configs_3) = 1, 20, 1, J1=-0.0010416666666666667
len(configs_1), len(configs_2), len(configs_3) = 1, 20, 1, J1=-0.0010309278350515464
len(configs_1), len(configs_2), len(configs_3) = 1, 20, 1, J1=-0.0010204081632653062
len(configs_1), len(configs_2), len(configs_3) = 1, 20, 1, J1=-0.00101010101010101
len(configs_1), len(configs_2), len(configs_3) = 1, 20, 1, J1=-0.001


In [410]:
# diagH

In [293]:
# def QUBO_program(): # adiabatic program
#     pass